# <center><b><font color="red">PyTorch Going Modular</font></b></center>

## **Script Mode**

Putting `%%writefile filename` at the top of a cell in Jupyter or Google Colab will write the contents of that cell to a specified `filename`.

> Since using **Jupyter/Google Colab notebooks** is a popular way of starting off data science and machine learning projects, knowing about the `%%writefile` magic command is a handy tip.

In [1]:
import os
import torch

In [2]:
# Create a Python file called hello_world.py
%%writefile hello_world.py
print("I Love PyTorch")

Writing hello_world.py


In [3]:
# Run this Python file
!python hello_world.py

I Love PyTorch


In [4]:
! rm -rf hello_world.py

## **Directory Structure**

⭐ **PyTorch project structure:**
```
pytorch_project/
├── pytorch_project/
│   ├── data_setup.py
│   ├── engine.py
│   ├── model.py
│   ├── train.py
│   └── utils.py
├── models/
│   ├── model_1.pth
│   └── model_2.pth
└── data/
    ├── data_folder_1/
    └── data_folder_2/
```

⭐ `TinyVGG` ~ `pizza_steak_sushi`
```
going_modular/
├── going_modular/
│   ├── data_setup.py
│   ├── engine.py
│   ├── model_builder.py
│   ├── train.py
│   └── utils.py
├── models/
│   ├── 05_going_modular_cell_mode_tinyvgg_model.pth
│   └── 05_going_modular_script_mode_tinyvgg_model.pth
└── data/
    └── pizza_steak_sushi/
        ├── train/
        │   ├── pizza/
        │   │   ├── image01.jpeg
        │   │   └── ...
        │   ├── steak/
        │   └── sushi/
        └── test/
            ├── pizza/
            ├── steak/
    

## **1. Get data**

In [5]:
import os

os.makedirs("going_modular", exist_ok=True)

In [6]:
import requests
import zipfile
from pathlib import Path

# Step 1: Set up the path
url = "https://github.com/mohd-faizy/PyTorch-Essentials/raw/main/_datasets/pizza_steak_sushi.zip"
image_path = Path("data/pizza_steak_sushi")
image_path.mkdir(parents=True, exist_ok=True)
zip_path = image_path.parent / "pizza_steak_sushi.zip"

# Step 2: Download the zip file containing the data
zip_path.write_bytes(requests.get(url).content)

# # Step 3: Unzip the downloaded file into the image directory
with zipfile.ZipFile(zip_path, "r") as zip_ref:
    zip_ref.extractall(image_path)

In [7]:
!ls /content/data

pizza_steak_sushi  pizza_steak_sushi.zip


In [8]:
# setup train and testing paths
train_dir = image_path / "train"
test_dir = image_path / "test"

train_dir, test_dir

(PosixPath('data/pizza_steak_sushi/train'),
 PosixPath('data/pizza_steak_sushi/test'))

## **2. Create Datasets and DataLoaders**

`transforms`  → `ImageFolder(datasets)` → `DataLoader`

In [9]:
%%writefile going_modular/data_setup.py
"""
Provides functionality for creating PyTorch DataLoaders for
image classification tasks.
"""
import os

from torch.utils.data import DataLoader
from torchvision import datasets, transforms

NUM_WORKERS = os.cpu_count()

def create_dataloaders(
    train_dir: str,
    test_dir: str,
    transform: transforms.Compose,
    batch_size: int,
    num_workers: int = NUM_WORKERS
):
    """Generates training and testing DataLoaders from directory paths.

    Converts the specified training and testing directories into
    PyTorch Datasets and then into DataLoaders.

    Args:
        train_dir (str): Path to the training data directory.
        test_dir (str): Path to the testing data directory.
        transform (transforms.Compose): Transformations to apply to the data.
        batch_size (int): Number of samples per batch in each DataLoader.
        num_workers (int, optional): Number of subprocesses to use for data loading.
                                     Defaults to the number of CPU cores.

    Returns:
        tuple: Contains (train_dataloader, test_dataloader, class_names).
            - train_dataloader (DataLoader): DataLoader for the training data.
            - test_dataloader (DataLoader): DataLoader for the testing data.
            - class_names (list): List of class names from the target dataset.

    Example:
        train_dataloader, test_dataloader, class_names = create_dataloaders(
            train_dir="path/to/train_dir",
            test_dir="path/to/test_dir",
            transform=some_transform,
            batch_size=32,
            num_workers=4
        )
    """

    # Create datasets from image folders
    train_data = datasets.ImageFolder(train_dir,
                                      transform=transform)
    test_data = datasets.ImageFolder(test_dir,
                                     transform=transform)

    # Extract class names
    class_names = train_data.classes

    # Initialize DataLoaders
    train_dataloader = DataLoader(
        train_data,
        batch_size=batch_size,
        shuffle=True,
        num_workers=num_workers,
        pin_memory=True,
    )
    test_dataloader = DataLoader(
        test_data,
        batch_size=batch_size,
        shuffle=False,
        num_workers=num_workers,
        pin_memory=True,
    )

    return train_dataloader, test_dataloader, class_names

Writing going_modular/data_setup.py


In [10]:
import torch
from going_modular import data_setup
from torchvision import transforms

# Create simple transform
data_transform = transforms.Compose([
    transforms.Resize((64, 64)),
    transforms.ToTensor()
])

NUM_WORKERS = os.cpu_count()

train_dataloader, test_dataloader, class_names = data_setup.create_dataloaders(
    train_dir=train_dir,
    test_dir=test_dir,
    transform=data_transform,
    batch_size=1,
    num_workers=os.cpu_count()
)

image, label = next(iter(train_dataloader))
image.shape, label.shape

(torch.Size([1, 3, 64, 64]), torch.Size([1]))

## **3. Making a model (TinyVGG)**




In [11]:
%%writefile going_modular/model_builder.py
"""
Provides PyTorch model code for instantiating a TinyVGG model.
"""
import torch
from torch import nn

class TinyVGG(nn.Module):
    """Defines the TinyVGG architecture.

    This class replicates the TinyVGG architecture as described on the CNN explainer website.
    The original architecture can be found here: https://poloclub.github.io/cnn-explainer/

    Args:
        input_shape (int): Number of input channels.
        hidden_units (int): Number of hidden units between layers.
        output_shape (int): Number of output units.
    """
    def __init__(self, input_shape: int, hidden_units: int, output_shape: int) -> None:
        super().__init__()
        self.conv_block_1 = nn.Sequential(
            nn.Conv2d(in_channels=input_shape,
                      out_channels=hidden_units,
                      kernel_size=3,
                      stride=1,
                      padding=0),
            nn.ReLU(),
            nn.Conv2d(in_channels=hidden_units,
                      out_channels=hidden_units,
                      kernel_size=3,
                      stride=1,
                      padding=0),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2)
        )
        self.conv_block_2 = nn.Sequential(
            nn.Conv2d(hidden_units, hidden_units, kernel_size=3, padding=0),
            nn.ReLU(),
            nn.Conv2d(hidden_units, hidden_units, kernel_size=3, padding=0),
            nn.ReLU(),
            nn.MaxPool2d(2)
        )
        self.classifier = nn.Sequential(
            nn.Flatten(),
            nn.Linear(in_features=hidden_units * 13 * 13, out_features=output_shape)
        )

    def forward(self, x: torch.Tensor) -> torch.Tensor:
        x = self.conv_block_1(x)
        x = self.conv_block_2(x)
        x = self.classifier(x)
        return x
        # return self.classifier(self.conv_block_2(self.conv_block_1(x))) # <- leverage the benefits of operator fusion

Writing going_modular/model_builder.py


In [12]:
import torch

from going_modular import model_builder

device = "cuda" if torch.cuda.is_available() else "cpu"

# Instantiate an instance of the model from the "model_builder.py" script
torch.manual_seed(42)
model = model_builder.TinyVGG(input_shape=3, # number of color channels (3 for RGB)
                              hidden_units=10,
                              output_shape=len(class_names)).to(device)
model

TinyVGG(
  (conv_block_1): Sequential(
    (0): Conv2d(3, 10, kernel_size=(3, 3), stride=(1, 1))
    (1): ReLU()
    (2): Conv2d(10, 10, kernel_size=(3, 3), stride=(1, 1))
    (3): ReLU()
    (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (conv_block_2): Sequential(
    (0): Conv2d(10, 10, kernel_size=(3, 3), stride=(1, 1))
    (1): ReLU()
    (2): Conv2d(10, 10, kernel_size=(3, 3), stride=(1, 1))
    (3): ReLU()
    (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (classifier): Sequential(
    (0): Flatten(start_dim=1, end_dim=-1)
    (1): Linear(in_features=1690, out_features=3, bias=True)
  )
)

**Do a dummy forward pass on model.**

In [13]:
# 1. Get a batch of images and labels from the DataLoader
img_batch, label_batch = next(iter(train_dataloader))

# 2. Get a single image from the batch and unsqueeze the image so its shape fits the model
img_single, label_single = img_batch[0].unsqueeze(dim=0), label_batch[0]
print(f"Single image shape: {img_single.shape}\n")

# 3. Perform a forward pass on a single image
model.eval()
with torch.inference_mode():
    pred = model(img_single.to(device))

# 4. Print out what's happening and convert model logits -> pred probs -> pred label
print(f"Output logits:\n{pred}\n")
print(f"Output prediction probabilities:\n{torch.softmax(pred, dim=1)}\n")
print(f"Output prediction label:\n{torch.argmax(torch.softmax(pred, dim=1), dim=1)}\n")
print(f"Actual label:\n{label_single}")

Single image shape: torch.Size([1, 3, 64, 64])

Output logits:
tensor([[ 0.0208, -0.0020,  0.0095]])

Output prediction probabilities:
tensor([[0.3371, 0.3295, 0.3333]])

Output prediction label:
tensor([0])

Actual label:
0


## **4. Creating `train_step()` and `test_step()` functions and `train()` to combine them.**

In [14]:
%%writefile going_modular/engine.py
"""
Contains functions for training and testing a PyTorch model.
"""

from typing import Dict, List, Tuple
import torch
from tqdm.auto import tqdm

def train_step(model: torch.nn.Module,
               dataloader: torch.utils.data.DataLoader,
               loss_fn: torch.nn.Module,
               optimizer: torch.optim.Optimizer,
               device: torch.device) -> Tuple[float, float]:
    """Trains a PyTorch model for a single epoch.

    Args:
        model: A PyTorch model to be trained.
        dataloader: A DataLoader instance for the model to be trained on.
        loss_fn: A PyTorch loss function to minimize.
        optimizer: A PyTorch optimizer to help minimize the loss function.
        device: A target device to compute on (e.g., "cuda" or "cpu").

    Returns:
        A tuple of training loss and training accuracy metrics in the form
        (train_loss, train_accuracy).
    """
    # Set model to training mode
    model.train()

    # Initialize training loss and accuracy
    train_loss, train_acc = 0, 0

    # Loop through data batches
    for batch, (X, y) in enumerate(dataloader):
        # Transfer data to target device
        X, y = X.to(device), y.to(device)

        # Forward pass
        y_pred = model(X)

        # Calculate and accumulate loss
        loss = loss_fn(y_pred, y)
        train_loss += loss.item()

        # Zero gradients
        optimizer.zero_grad()

        # Backward pass
        loss.backward()

        # Optimizer step
        optimizer.step()

        # Calculate and accumulate accuracy
        y_pred_class = torch.argmax(torch.softmax(y_pred, dim=1), dim=1)
        train_acc += (y_pred_class == y).sum().item() / len(y_pred)

    # Calculate average loss and accuracy per batch
    train_loss /= len(dataloader)
    train_acc /= len(dataloader)
    return train_loss, train_acc

def test_step(model: torch.nn.Module,
              dataloader: torch.utils.data.DataLoader,
              loss_fn: torch.nn.Module,
              device: torch.device) -> Tuple[float, float]:
    """Tests a PyTorch model for a single epoch.

    Args:
        model: A PyTorch model to be tested.
        dataloader: A DataLoader instance for the model to be tested on.
        loss_fn: A PyTorch loss function to calculate loss on the test data.
        device: A target device to compute on (e.g., "cuda" or "cpu").

    Returns:
        A tuple of testing loss and testing accuracy metrics in the form
        (test_loss, test_accuracy).
    """
    # Set model to evaluation mode
    model.eval()

    # Initialize test loss and accuracy
    test_loss, test_acc = 0, 0

    # Disable gradient calculations for inference
    with torch.inference_mode():
        # Loop through data batches
        for batch, (X, y) in enumerate(dataloader):
            # Transfer data to target device
            X, y = X.to(device), y.to(device)

            # Forward pass
            test_pred_logits = model(X)

            # Calculate and accumulate loss
            loss = loss_fn(test_pred_logits, y)
            test_loss += loss.item()

            # Calculate and accumulate accuracy
            test_pred_labels = test_pred_logits.argmax(dim=1)
            test_acc += (test_pred_labels == y).sum().item() / len(test_pred_labels)

    # Calculate average loss and accuracy per batch
    test_loss /= len(dataloader)
    test_acc /= len(dataloader)
    return test_loss, test_acc

def train(model: torch.nn.Module,
          train_dataloader: torch.utils.data.DataLoader,
          test_dataloader: torch.utils.data.DataLoader,
          optimizer: torch.optim.Optimizer,
          loss_fn: torch.nn.Module,
          epochs: int,
          device: torch.device) -> Dict[str, List[float]]:
    """Trains and tests a PyTorch model over multiple epochs.

    Args:
        model: A PyTorch model to be trained and tested.
        train_dataloader: A DataLoader instance for training data.
        test_dataloader: A DataLoader instance for testing data.
        optimizer: A PyTorch optimizer to help minimize the loss function.
        loss_fn: A PyTorch loss function to calculate loss on both datasets.
        epochs: Number of epochs to train for.
        device: A target device to compute on (e.g., "cuda" or "cpu").

    Returns:
        A dictionary of training and testing metrics, each containing a list
        of values for each epoch. Format:
        {
            "train_loss": [...],
            "train_acc": [...],
            "test_loss": [...],
            "test_acc": [...]
        }
    """
    # Initialize results dictionary
    results = {
        "train_loss": [],
        "train_acc": [],
        "test_loss": [],
        "test_acc": []
    }

    # Loop through epochs
    for epoch in tqdm(range(epochs)):
        # Training step
        train_loss, train_acc = train_step(
            model=model,
            dataloader=train_dataloader,
            loss_fn=loss_fn,
            optimizer=optimizer,
            device=device
        )

        # Testing step
        test_loss, test_acc = test_step(
            model=model,
            dataloader=test_dataloader,
            loss_fn=loss_fn,
            device=device
        )

        # Print progress
        print(
            f"Epoch: {epoch+1} | "
            f"train_loss: {train_loss:.4f} | "
            f"train_acc: {train_acc:.4f} | "
            f"test_loss: {test_loss:.4f} | "
            f"test_acc: {test_acc:.4f}"
        )

        # Update results dictionary
        results["train_loss"].append(train_loss)
        results["train_acc"].append(train_acc)
        results["test_loss"].append(test_loss)
        results["test_acc"].append(test_acc)

    # Return results
    return results

Writing going_modular/engine.py


## **5. Creating a function to save the model**

In [15]:
%%writefile going_modular/utils.py
"""
Contains various utility functions for PyTorch model training and saving.
"""
import os
import torch
from pathlib import Path

def save_model(model: torch.nn.Module,
               target_dir: str,
               model_name: str):
    """
    Saves a PyTorch model to a specified directory.

    Args:
        model (torch.nn.Module): The PyTorch model to be saved.
        target_dir (str): The directory where the model will be saved.
        model_name (str): The name of the saved model file. Must include ".pth" or ".pt" as the file extension.

    Example:
        save_model(model=model_0,
        target_dir="models",
        model_name="05_going_modular_tingvgg_model.pth")
    """
    # Create the target directory if it doesn't exist
    target_dir_path = Path(target_dir)
    target_dir_path.mkdir(parents=True, exist_ok=True)

    # Ensure the model name has the correct file extension
    assert model_name.endswith(".pth") or model_name.endswith(".pt"), "model_name must end with '.pt' or '.pth'"

    # Define the full path for the saved model
    model_save_path = target_dir_path / model_name

    # Save the model's state dictionary
    print(f"[INFO] Saving model to: {model_save_path}")
    torch.save(obj=model.state_dict(),
               f=model_save_path)

Writing going_modular/utils.py


## **6. Train, evaluate and save the model**

In [16]:
%%writefile going_modular/train.py
"""
Trains a PyTorch image classification model using device-agnostic code.
"""

import os
import torch
from torchvision import transforms
import data_setup, engine, model_builder, utils


# Setup hyperparameters
NUM_EPOCHS = 10
BATCH_SIZE = 32
HIDDEN_UNITS = 10
LEARNING_RATE = 0.001

# Setup directories
train_dir = "data/pizza_steak_sushi/train"
test_dir = "data/pizza_steak_sushi/test"

# Setup target device
device = "cuda" if torch.cuda.is_available() else "cpu"

# Create transforms
data_transform = transforms.Compose([
  transforms.Resize((64, 64)),
  transforms.ToTensor()
])

# 1. Create DataLoaders with help from `data_setup.py`--------------------------
train_dataloader, test_dataloader, class_names = data_setup.create_dataloaders(
    train_dir=train_dir,
    test_dir=test_dir,
    transform=data_transform,
    batch_size=BATCH_SIZE
)

# 2. Create model with help from `model_builder.py`-----------------------------
model = model_builder.TinyVGG(
    input_shape=3,
    hidden_units=HIDDEN_UNITS,
    output_shape=len(class_names)
).to(device)

# Set loss and optimizer
loss_fn = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(),
                             lr=LEARNING_RATE)

# 3. Start training with help from `engine.py`----------------------------------
engine.train(model=model,
             train_dataloader=train_dataloader,
             test_dataloader=test_dataloader,
             loss_fn=loss_fn,
             optimizer=optimizer,
             epochs=NUM_EPOCHS,
             device=device)

# 4. Save the model with help from `utils.py`-----------------------------------
utils.save_model(model=model,
                 target_dir="models",
                 model_name="05_going_modular_script_mode_tinyvgg_model.pth")

Writing going_modular/train.py


Now our final directory structure looks like:
```
data/
  └── pizza_steak_sushi/
      ├── train/
      │   ├── pizza/
      │   │   ├── train_image_01.jpeg
      │   │   ├── train_image_02.jpeg
      │   │   └── ...
      │   ├── steak/
      │   └── sushi/
      └── test/
          ├── pizza/
          │   ├── test_image_01.jpeg
          │   ├── test_image_02.jpeg
          │   └── ...
          ├── steak/
          └── sushi/
going_modular/
  ├── data_setup.py
  ├── engine.py
  ├── model_builder.py
  ├── train.py
  └── utils.py
models/
  └── saved_model.pth

```

Now to put it all together!

Let's run our `train.py` file from the command line with:

```
!python going_modular/train.py
```

In [17]:
# 5. Train----------------------------------------------------------------------
!python going_modular/train.py

  0% 0/10 [00:00<?, ?it/s]Epoch: 1 | train_loss: 1.1029 | train_acc: 0.2891 | test_loss: 1.0995 | test_acc: 0.2604
 10% 1/10 [00:02<00:26,  2.91s/it]Epoch: 2 | train_loss: 1.0940 | train_acc: 0.4375 | test_loss: 1.0818 | test_acc: 0.5417
 20% 2/10 [00:04<00:18,  2.34s/it]Epoch: 3 | train_loss: 1.1077 | train_acc: 0.2812 | test_loss: 1.0711 | test_acc: 0.5417
 30% 3/10 [00:06<00:14,  2.11s/it]Epoch: 4 | train_loss: 1.1007 | train_acc: 0.2812 | test_loss: 1.0778 | test_acc: 0.5521
 40% 4/10 [00:08<00:12,  2.01s/it]Epoch: 5 | train_loss: 1.0838 | train_acc: 0.4844 | test_loss: 1.0801 | test_acc: 0.4848
 50% 5/10 [00:10<00:09,  1.93s/it]Epoch: 6 | train_loss: 1.0770 | train_acc: 0.4727 | test_loss: 1.0700 | test_acc: 0.4328
 60% 6/10 [00:12<00:07,  1.89s/it]Epoch: 7 | train_loss: 1.0534 | train_acc: 0.4805 | test_loss: 1.0553 | test_acc: 0.3419
 70% 7/10 [00:14<00:06,  2.15s/it]Epoch: 8 | train_loss: 1.0071 | train_acc: 0.4961 | test_loss: 1.0454 | test_acc: 0.3930
 80% 8/10 [00:17<00:04, 